In [1]:
import pyspark
from pyspark.sql import SparkSession

In [59]:
pyspark.__version__

'3.3.2'

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/08 07:37:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-08 07:43:02--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
52.85.39.153, 52.85.39.117, 52.85.39.97, ...ci6vzurychx.cloudfront.net)... 
connected. to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.153|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  19.4MB/s    in 3.3s    

2025-03-08 07:43:06 (18.5 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [14]:
!wc -l yellow_tripdata_2024-10.parquet

254534 yellow_tripdata_2024-10.parquet


In [15]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [16]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [20]:
df = df.repartition(4)

In [21]:
df.write.parquet('yellow/4/2024/10/')

In [22]:
df = spark.read.parquet('yellow/4/2024/10/')

In [23]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [24]:
from pyspark.sql import functions as F

In [25]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-10 10:34:57|  2024-10-10 10:53:48|              1|          3.4|         1|                 N|         140|          74|           1|       20.5|  0.0|    0.5|      0.2

In [30]:
df \
    .withColumn('tpep_pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .select('tpep_pickup_date', 'PULocationID', 'DOLocationID') \
    .show()

+----------------+------------+------------+
|tpep_pickup_date|PULocationID|DOLocationID|
+----------------+------------+------------+
|      2024-10-10|         140|          74|
|      2024-10-08|         166|         107|
|      2024-10-05|          79|         255|
|      2024-10-04|          48|         263|
|      2024-10-01|         142|         239|
|      2024-10-05|         262|         107|
|      2024-10-05|         236|         239|
|      2024-10-07|         236|         113|
|      2024-10-02|         234|         143|
|      2024-10-02|         143|         246|
|      2024-10-02|         239|         142|
|      2024-10-10|         161|         237|
|      2024-10-04|         125|         186|
|      2024-10-03|         140|         229|
|      2024-10-05|         161|         162|
|      2024-10-04|          79|         141|
|      2024-10-04|         142|          43|
|      2024-10-10|         237|         229|
|      2024-10-04|         163|         142|
|      202

In [36]:
df \
    .withColumn('tpep_pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .groupBy('tpep_pickup_date') \
    .count() \
    .show()

[Stage 21:===========================================>              (3 + 1) / 4]

+----------------+------+
|tpep_pickup_date| count|
+----------------+------+
|      2024-10-24|137337|
|      2024-10-02|113906|
|      2009-01-01|     1|
|      2024-10-25|136066|
|      2024-10-22|121106|
|      2024-10-18|133213|
|      2024-10-08|121402|
|      2024-10-10|143206|
|      2024-10-20|117129|
|      2024-10-01|119118|
|      2024-10-04|112431|
|      2024-10-15|128893|
|      2024-10-28|110595|
|      2024-10-29|126931|
|      2024-10-17|136330|
|      2024-10-31|129394|
|      2024-10-07|102014|
|      2024-11-01|    26|
|      2024-10-11|128821|
|      2024-10-16|134891|
+----------------+------+
only showing top 20 rows



In [26]:
df.registerTempTable('trips_data')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [39]:
df_trips_count = spark.sql("""
SELECT 
    COUNT(1) AS number_records
FROM
    trips_data
WHERE
    tpep_pickup_datetime >= '2024-10-15 00:00:00' and
    tpep_pickup_datetime < '2024-10-16 00:00:00'
""").show()

+--------------+
|number_records|
+--------------+
|        128893|
+--------------+



In [45]:
df_longest_trip = spark.sql("""
SELECT 
    (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600.0 AS trip_hours
FROM 
    trips_data
ORDER BY 
    trip_hours DESC
LIMIT 1
""").show()

[Stage 32:>                                                         (0 + 4) / 4]

+----------+
|trip_hours|
+----------+
|162.617778|
+----------+



In [47]:
df_locations = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [48]:
df_locations.write.parquet('zones')

AnalysisException: path file:/home/ubuntu/data/data-engineering-zoomcamp/05-batch/code/zones already exists.

In [49]:
df_locations.registerTempTable('locations')

In [58]:
least_frequent_zone = spark.sql("""
SELECT 
    z.Zone AS pickup_zone, 
    COUNT(t.PULocationID) AS pickup_count
FROM trips_data t
JOIN locations z
ON t.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY pickup_count ASC
LIMIT 1
""").show()

+--------------------+------------+
|         pickup_zone|pickup_count|
+--------------------+------------+
|Governor's Island...|           1|
+--------------------+------------+

